In [2]:
import logging
import os
import sys
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np

from models.resnet import resnet18 

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# print_config()



2025-07-09 12:04:23.673282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752080664.298062   44861 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752080664.545819   44861 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-09 12:04:26.514288: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### **1. ResNet18: El Caballo de Batalla Generalista**

**ResNet18** es una arquitectura de red neuronal convolucional muy popular, parte de la familia Residual Networks (ResNet). Su innovación principal radica en el uso de **"bloques residuales"** o "saltos" (skip connections). Estos saltos permiten que la información y los gradientes fluyan más fácilmente a través de muchas capas, resolviendo el problema del "gradiente desvanecido" que dificultaba el entrenamiento de redes muy profundas.

* **Pre-entrenamiento:** Comúnmente, ResNet18 se pre-entrena en el dataset **ImageNet**. Este es un gigantesco conjunto de datos con millones de imágenes de objetos cotidianos (perros, coches, sillas, etc.) de 1000 categorías diferentes.
* **Ventajas en imágenes médicas:** A pesar de haber sido entrenado con imágenes naturales, las características de bajo nivel que ResNet18 aprende de ImageNet (detección de bordes, patrones de textura) son sorprendentemente útiles como punto de partida para el análisis de imágenes médicas. Es un excelente punto de inicio general.
* **Consideración:** Puede haber una "brecha de dominio" entre las imágenes de ImageNet y las imágenes médicas, lo que significa que el fine-tuning es crucial para adaptar el modelo a las particularidades de los datos médicos (contrastes, resoluciones, tipos de ruido).

---

#### **2. MedicalNet: Especialización para el Dominio Médico**

**MedicalNet** es una iniciativa que proporciona modelos (incluyendo variantes de ResNet como ResNet18) que han sido **pre-entrenados específicamente en un vasto y diverso conjunto de datos de imágenes médicas**.

* **Pre-entrenamiento:** A diferencia de ResNet18 estándar, MedicalNet ha sido entrenado con millones de imágenes provenientes de diversas modalidades médicas (MRI, CT, rayos X, ultrasonido) y cubriendo diferentes órganos y patologías.
* **Ventajas en imágenes médicas:**
    * **Mayor relevancia de las características:** Al estar pre-entrenado en datos médicos, MedicalNet ya ha aprendido patrones y características que son intrínsecamente más relevantes para el diagnóstico y análisis clínico.
    * **Menor brecha de dominio:** Esto puede traducirse en un mejor rendimiento inicial, una convergencia más rápida durante el fine-tuning y, potencialmente, un mejor rendimiento final con menos datos de entrenamiento específicos para tu tarea.
    * **Adaptado a diferentes modalidades:** Su entrenamiento diverso lo hace robusto para trabajar con distintos tipos de imágenes médicas.



In [3]:
# Dimensiones de tus volúmenes de MRI usamos 256x256x256 ya que es un tamaño común para imágenes de resonancia magnética cerebral en freesurfer
input_D = 256
input_H = 256
input_W = 256

# Número de canales (1 para norm.mgz)
input_C = 1 
model = resnet18(sample_input_D=input_D,
                 sample_input_H=input_H,
                 sample_input_W=input_W,
                 num_seg_classes=1)  # ATENTO A ESTO PORQUE ES IMPORTANTE PARA LA CLASIFICACION

/home/nicolas/Documentos/gliara/models/resnet.py:173: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


Pesos de MedicalNet y ResNet18:
https://share.weiyun.com/55sZyIx 

In [13]:
pretrained_weights_path = "pretrain/resnet_18_23dataset.pth"

try:
    state_dict = torch.load(pretrained_weights_path)
    model.load_state_dict(state_dict, strict=False)
    print(f"Pesos de ResNet-18 cargados exitosamente desde {pretrained_weights_path}")

except FileNotFoundError:
    print(f"Error: El archivo de pesos no se encontró en {pretrained_weights_path}")
    print("Asegúrate de que la ruta sea correcta y el archivo exista.")
except Exception as e:
    print(f"Ocurrió un error al cargar los pesos: {e}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Modelo movido a: {device}")

Pesos de ResNet-18 cargados exitosamente desde pretrain/resnet_18_23dataset.pth
Modelo movido a: cuda


/tmp/ipykernel_44861/2331833451.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_weights_path)


In [14]:
# Define el número de clases para tu tarea de clasificación binaria (Alzheimer sí/no)
your_num_classes = 2
final_conv_layer = model.conv_seg[0]
num_in_features = final_conv_layer.in_channels
model.conv_seg = nn.Sequential(
    nn.Conv3d(num_in_features, your_num_classes,
              kernel_size=final_conv_layer.kernel_size,
              stride=final_conv_layer.stride,
              padding=final_conv_layer.padding)
)

print(f"Capa de salida (conv_seg) adaptada a {your_num_classes} clases para el proyecto Gliara.")

# Mueve el modelo completo (incluida la nueva capa) a la GPU
model.to(device)
print(f"Modelo movido a: {device}")


Capa de salida (conv_seg) adaptada a 2 clases para el proyecto Gliara.
Modelo movido a: cuda


# Preparar la data